**Вопросы для повторения:**
* что такое `std::mutex`? зачем он нужен? что внутри?
* что такое `std::lock_guard`? чем не устраивают `std::mutex::lock`, `std::mutex::unlock`?
* предположим, у нас есть многопоточная очередь задач `MTTasksQueue`. В чём здесь проблема? Как её будем исправлять?

```c++
void run_queued_task(MTTasksQueue& q)
{
    if (!q.empty())
        run_task(q.pop());
}
```

* а в чём может быть проблема с той же самой очередью тут? Какие есть варианты её исправить?

```c++
class MTTasksQueue
{
public:
    using Task = std::function<void(void)>;
    
    ...

    void pop_and_run()
    {
        std::lock_guard guard(mtx);

        if (!tasks_queue.empty())
        {        
            const auto task = std::move(tasks_queue.back());
            tasks_queue.pop_back();
            task();
        }        
    }

private:
    std::mutex mtx;
    std::queue<Task> tasks_queue;
};
```

<details>
<summary>ответ</summary>
<p>recursive_mutex и реорганизация кода. Что такое и как устроен recursive_mutex? Когда его следует использовать? Почему в данном случае recursive_mutex - плохое решение? Если остаться на обычном mutex, как следует с ним поступить?</p>
</details>

* Что такое deadlock? Каким минимальным числом потоков и mutex-ов устроить deadlock?

<details>
<summary>замечание</summary>
<p>"на самом деле", дважды вызов lock у std::mutex на одном потоке - это не обязательно deadlock. Документация утверждает, что это UB, и _скорее всего_ вы получите deadlock, но некоторые реализации могут задетектить ситуацию и бросить exception, но в общем случае всё совсем плохо - это UB</p>
<p><a href="https://en.cppreference.com/w/cpp/thread/mutex/lock">proof</a></p>
</details>


* что такое и зачем нужны `shared_mutex`, `unique_lock`, `shared_lock` ?

<br />

**Пятиминутка:**

Найдите ошибки и недочёты. Укажите как их поправить.

```c++
class TasksQueue
{
    std::deque<Task> tasks;
    std::mutex mtx;
    std::condition_variable task_appeared_cv;
    
public:
    void push(Task task)
    {
        std::lock_guard guard(mtx);
        tasks.emplace_back(std::move(task));
        
        task_appeared_cv.notify_all();
    }
    
    Task pop()
    {
        {
            std::lock_guard guard(mtx);
            if (!tasks.empty())
                return pop_task_unsafe();
        }
        
        std::unique_lock lk(mtx);
        task_appeared_cv.wait(lk);
        return pop_task_unsafe();
    }

private:
    Task pop_task_unsafe()
    {
        auto t = std::move(tasks.back());
        tasks.pop_back();
        return t;
    }    
};
```

<details>
<summary>ответ</summary>
<p>

1. `notify_all` -> `notify_one` (т.к. появилась только одна задача, достаточно разбудить только один ждущий задач поток)
2. мьютекс желательно иметь разлоченным перед вызовом `notify_all`/`notify_one` (почему? как реализовать?)
3. `task_appeared_cv.wait(lk);` подвержен spurious wakeups (что это такое? как чинить?)
4. в случае с `notify_all` первый пробудившийся по `wait` выхватит элемент из очереди, остальные увидят пустую очередь и словят UB.
5. [опицонально] вызов `pop` - блокирующий, пока не появится задача в очереди. Это допустимо, если есть гарантия, что задача в очереди точно появится. Если гарантии нет, то нужно менять дизайн класса (какие варианты?)
    
</p>
</details>